In [75]:
import random

def BSC(p_err, x):
    if p_err < 0 or p_err >= 1:
        print("p_err must be in [0,1[")
    r = random.random()
    if r < p_err:
        return 1-x
    return x

def e(i, n):
    v = vector(GF(2), [0]*n)
    v[i] = 1
    return v

def decode_bit_flipping(H, Y, iter_max):
    n = H.ncols()
    k = n - H.nrows()
    min_weight_s = (H*Y).hamming_weight()
    t = 0
    while (H*Y).hamming_weight() != 0 and t < iter_max:
        s_weight = (H*Y).hamming_weight()
        
        #print("Poids syndrôme : " ,s_weight)
        changes = [0 for i in range(n)]
        for i in range(n):
            w = (H*(Y + e(i, n))).hamming_weight()
            #print("w:",w)
            if  w <= min_weight_s and w < s_weight:
                if w < min_weight_s:
                    for j in range(n):
                        changes[j] = 0
                    min_weight_s = w
                changes[i] = 1
        changes = vector(GF(2), changes)
        #print(changes)
        Y = Y + changes
        t += 1
    print("Nb iter : ", t)
    return Y

#retourne V et W où V[i] contiendra les indices des noeuds de parité voisins du noeud d'information i
# et W[i] contiendra les indices des noeuds d'information voisins du noeud de parité j
def voisins(H):
    V = []
    W = []
    for i in range(H.ncols()):
        #print("i : ",i)
        tmp = []
        for j in range(H.nrows()):
            #print("    j : ", j)
            if H[j,i] == 1:
                tmp.append(j)
        V.append(tmp)
    for i in range(H.nrows()):
        tmp = []
        for j in range(H.ncols()):
            if H[i,j] == 1:
                tmp.append(j)
        W.append(tmp)
    return V,W
    
def decode_BP(H, y, p_err_canal, iter_max):
    if (H*y).hamming_weight() == 0:
        return y
    n = H.ncols()
    k = n - H.nrows()
    V,W = voisins(H)
    #Init des vecteur Q et R
    Q = Matrix(RR, n, n-k, sparse = True)
    R = Matrix(RR, n-k, n, sparse = True)
    for i in range(n):
        for j in range(n-k):
            if y[i] == 0:
                Q[i,j] = p_err_canal
            else:
                Q[i,j] = 1 - p_err_canal
    #print(Q)
    #print(R)
    cmptr = 0
    while cmptr < iter_max:
        #étape horizontale
        for j in range(n-k):
            for i in W[j]:
                #calcul de R_{j,i}
                p_tmp = 1
                for s in W[j]:
                    if s != i:
                        p_tmp = p_tmp * (1 - 2*Q[s,j])
                R[j,i] = 0.5 * ( 1 - p_tmp )
        
        #étape de décision
        x = []
        for i in range(n):
            p1 = p_err_canal
            p2 = 1 - p1
            for j in V[i]:
                p1 = p1 * R[j,i]
                p2 = p2 * (1 - R[j,i])
            q = p1 / (p1 + p2)
            if q > 0.5:
                x.append(1)
            else:
                x.append(0)
        x = vector(GF(2), x)
        
        if (H*x).hamming_weight() == 0:
            print("Nb iter : ", cmptr)
            return x
        cmptr += 1
        #étape verticale
        for i in range(n):
            for j in V[i]:
                #calcul de Q_{i,j}
                p1 = p_err_canal
                p2 = 1 - p1
                for s in V[i]:
                    if s != j:
                        p1 = p1 * R[s,i]
                        p2 = p2 * (1 - R[s,i])
                Q[i,j] = p1 / (p1 + p2)
    ''' version 1 : index out of range dans les q
        #étape horizontale
        for j in range(len(W)):
            l = len(W[j])
            for i in W[j]:
                #calcul de R_{j,i}
                p_tmp = 1
                for s in range(l):
                    if s != i:
                        p_tmp = p_tmp * (1 - 2*Q[s][j])
                R[j][i] = 0.5 * ( 1 - p_tmp )
        
        #étape de décision
        x = []
        for i in range(n):
            l = len(V[i])
            p1 = p_err_canal
            p2 = 1 - p1
            for j in range(l):
                p1 = p1 * R[j][i]
                p2 = p2 * (1 - R[j][i])
            q = p1 / (p1 + p2)
            if q > 0.5:
                x.append(1)
            else:
                x.append(0)
        x = vector(GF(2), x)
        
        if (H*x).hamming_weight() == 0:
            return x
        
        #étape verticale
        for i in range(n):
            l = len(V[i])
            for j in range(l):
                #calcul de Q_{i,j}
                p1 = p_err_canal
                p2 = 1 - p1
                for s in range(l):
                    if s != j:
                        p1 = p1 * R[s][i]
                        p2 = p2 * (1 - R[s][i])
                Q[i][j] = p1 / (p1 + p2)
        '''

In [51]:
#Extension d'une matrice de parité régulière de LDPC

h = [[1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
h = Matrix(GF(2), h)
print(h)
ncols = h.ncols()
nrows = h.nrows()
z = 3
I = Matrix(GF(2),z)
O = Matrix(GF(2), z, sparse = True)
for i in range(z):
    I[i, i] = 1
H = []

for i in range(nrows):
    tmp = []
    for j in range(ncols):
        if h[i,j] == 0:
            tmp.append(O)
        else :
            I2 = I.columns()
            random.shuffle(I2)
            I = Matrix(GF(2), I2)
            tmp.append(I)
    tmp = block_matrix(GF(2),1,ncols,tmp)
    if i == 0:
        H = tmp
    else:
        H = block_matrix(GF(2), 2,1, [H,tmp], subdivide = False)
    
#print(H)


[1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
[0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0]
[1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0]
[0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0]
[1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0]
[0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
[0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0]
[1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1]


In [411]:
nH = ncols * z
kH = nH - nrows * z
print("n : ", nH,"\nk : ", kH)

n :  60 
k :  30


In [452]:
C = H.right_kernel()
y = C.random_element()
print("y = ")
print(y)
print("Passage dans le canal binaire symétrique")
y2 = vector(GF(2), nH)
for i in range(nH):
    y2[i] = BSC(0.05,y[i])
print("y' en sortie de canal:")
print(y2)
#print("s = H*y = ", H*y2)
c = decode_bit_flipping(H, y2, 100)
#print("y et c ")
#print(y)
#print(c)
print("c in C H * c = ", H*c)
print("well decoded : ", c == y)

y = 
(0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1)
Passage dans le canal binaire symétrique
y' en sortie de canal:
(0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1)
Nb iter :  2
c in C H * c =  (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
well decoded :  True


In [21]:
h_test = [[1,1,1,0,0,0,0,0],
          [0,0,0,1,1,1,0,0],
          [1,0,0,1,0,0,1,0],
          [0,1,0,0,1,0,0,1]
         ]
H_test = Matrix(GF(2),h_test)
#print(H_test)
V, W = voisins(H_test)
#print("V :")
#print(V)
#print("W :")
#print(W)

V :
[[0, 2], [0, 3], [0], [1, 2], [1, 3], [1], [2], [3]]
W :
[[0, 1, 2], [3, 4, 5], [0, 3, 6], [1, 4, 7]]


In [76]:
y = vector(GF(2),[1,0,0,1,0,1,0,1])
p_err_canal = 0.1
iter_max = 100
decode_BP(H_test, y ,p_err_canal,iter_max)

Nb iter :  1


(0, 0, 0, 0, 0, 0, 0, 0)